In [1]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score

In [2]:
def preprocesar_imagen(ruta_imagen):
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is None:  # Manejar imágenes no cargadas
        print(f"Error al cargar la imagen: {ruta_imagen}")
        return None
    imagen = cv2.resize(imagen, (128, 128))  # Redimensionar a 128x128
    imagen = cv2.equalizeHist(imagen)        # Ecualización del histograma
    return imagen

In [3]:
def extraer_caracteristicas_hog(imagen):
    fd = hog(
        imagen,
        orientations=9,  # Aumentamos el número de orientaciones
        pixels_per_cell=(8, 8),  # Reducimos el tamaño de las celdas para capturar más detalles
        cells_per_block=(2, 2),  # Aumentamos el tamaño del bloque para mejorar la normalización
        visualize=False  # Solo devuelve el descriptor
    )
    return fd

In [4]:
def crear_dataset(ruta_imagenes, extractor_caracteristicas):
    """
    Crea un dataset de características y etiquetas a partir de un directorio.
    """
    X, y = [], []
    for etiqueta, nombre_clase in enumerate(['cat', 'dog']):
        directorio_clase = os.path.join(ruta_imagenes, nombre_clase)
        for nombre_archivo in os.listdir(directorio_clase):
            ruta_imagen = os.path.join(directorio_clase, nombre_archivo)
            imagen = preprocesar_imagen(ruta_imagen)
            if imagen is None:  # Omitir imágenes no válidas
                continue
            caracteristicas = extractor_caracteristicas(imagen)
            X.append(caracteristicas)
            y.append(etiqueta)
    return np.array(X), np.array(y)

In [5]:
def clasificacion(y_real, y_predicha):
    nombres_clases = {0: "gatos", 1: "perros"}

    # Inicializar un diccionario para almacenar las métricas por clase
    reporte = {}
    for clase in [0, 1]:
        # Calcular las métricas por clase
        precision = precision_score(y_real, y_predicha, labels=[clase], average="micro")
        reporte[nombres_clases[clase]] = {
            'precisión': precision
        }
    # Imprimir clasificación
    print("Clasificación:")
    for nombre_clase, metricas in reporte.items():
        print(f"\nClase {nombre_clase}:")
        for metrica, valor in metricas.items():
            print(f"  {metrica}: {valor:.2f}")

In [6]:
ruta_train = 'cat_dog_500/train'
ruta_test = 'cat_dog_500/test'

print("Extrayendo características HOG...")
X_train, y_train = crear_dataset(ruta_train, extraer_caracteristicas_hog)
X_test, y_test = crear_dataset(ruta_test, extraer_caracteristicas_hog)

print("Normalizando características...")
# Normalizamos las características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Realizando búsqueda de hiperparámetros con GridSearchCV...")
parametros = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}
clf = SVC(kernel='rbf')
busqueda_grid = GridSearchCV(clf, parametros, cv=5, scoring='accuracy')
busqueda_grid.fit(X_train, y_train)

# Evaluando el modelo con los mejores parámetros
mejor_modelo = busqueda_grid.best_estimator_
y_predicha = mejor_modelo.predict(X_test)
clasificacion(y_test, y_predicha)

Extrayendo características HOG...
Normalizando características...
Realizando búsqueda de hiperparámetros con GridSearchCV...
Clasificación:

Clase gatos:
  precisión: 0.99

Clase perros:
  precisión: 0.81
